Welcome to the Equator Evaluator! This notebook is designed to test state-of-the-art language models (LLMs) either locally or via API. We’ve chosen to use OpenRouter because it’s OpenAI-compatible and provides access to over 276 different models.

In addition, we can evaluate local Ollama-based models. With a bit of effort, you can adapt any model—local or remote—that follows the OpenAI API format. Keep in mind that evaluations on local models may run more slowly than those on remote API models, owing to your machine’s memory constraints. Although we run the Equator Evaluator locally, you can also host it on a remote server.

For our evaluations, we’ll use the OpenAI API to access OpenRouter models. We’ve tested the free models to ensure they work as expected. Remember, local evaluations may still be slower than using an external API. Our official model evaluations will be presented on our website. Meanwhile, we’ll maintain a private list of over 1,005 reasoning and logic questions to guarantee that our results remain unbiased.

Our tool is versatile enough to handle any QA evaluations—including legal, medical, or financial—by simply adding them to the *linguistic_benchmark.json* file. Our project focuses on identifying logical and reasoning shortcomings in LLMs to help strengthen their problem-solving abilities. We’ve found that LLMs can be easily tricked, so our goal is to track their progress until they truly match human-level capabilities.

Looking ahead, our next step is to incorporate vision into the Equator Evaluator. We’re also planning to release more advanced, locally-runnable reasoning models soon.








To get started, please follow these steps:

1. **Obtain Your OpenRouter Key**  
   Visit https://openrouter.ai/settings/keys to get your OpenRouter key.

2. **Add Funds to Your Account**  
   Make sure to add a few dollars to your account so you can use any of the models they provide. For more information, visit https://openrouter.ai/models.

3. **Create a .env File**  
   In your root directory, create a .env file with the following line:  
   ```
   OPENROUTER_KEY="<add your API key from OpenRouter>"
   ```

4. **Install Ollama Locally**  
   Since we will be using LLaMa 3.2 3b as our evaluator, please install Ollama locally. Note that this model can be changed, but if you do so, you will need to edit the line in the `auto_eval_bernard_llm_vector_db_remote_qa.py` file at line 385:  
   ```python
   response = self.generate_chat(
       model="llama3.2", messages=evaluator_system_prompt, stream=False
   )
   ```

5. **Download Ollama**  
   You can download Ollama from https://ollama.com/.

6. **Pull the LLaMa Model**  
   Run the following command to pull the latest LLaMa model:  
   ```bash
   ollama pull llama3.2:latest
   ```

7. **Run Ollama**  
   Finally, execute Ollama with the command:  
   ```bash
   ollama run llama3.2
   ``` 

Make sure to follow each step carefully to ensure everything is set up correctly!




 Make sure you create a new python virtual environment and activate it! Run the below cell once!

In [ ]:
%pip install -r requirements.txt 

## Imports just need to run it to but not an issue if you run it multiple times. 

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
import os
import re
import json
import requests
import chromadb
import time 
from loguru import logger

from charting import create_performance_chart
from utils import get_llm_stats, load_all_llm_answers_from_json
from openai import OpenAI

# import csv
import sqlite3
from datetime import datetime  # Correct import
import pandas as pd
from IPython.display import display
from auto_eval_bernard import Bernard_Controller, VectorDB_Controller, extract_model_parts


## User Instructions :  Variables
This section allows us to configure various configurations of the LLM Evaluator. For example if you just want to run the static analysis just comment out the llm_evaluate in the execution list.  You can also set the models you like to evaluate.  We are using OpenAI api call to the openrouter_models.  Change them to the models you like to evaluate.   Open router has about 275 models to choose.  They also have free models which are limited to about 200 calls per day. So you will need to create a paid account and use the none free models to avoid the limitation.  We have evaluated the free models just to test the code and make sure everything works as expected.



With respect to keepVectorDB you can set it to true to avoid imputing the data if you have already done it.  Please note that we input the data from linguistic_bechmark.json.  You are free to customize it for your purposes.  This data is the source of truth for our evaluator.  It is the answer key for grading the  "student".  

Also with respect to folder directory structures,  you can hard code the date which will keep using the same directory structure.   This section allows you to configure various settings for the LLM Evaluator. For instance, if you only want to run the static analysis, simply comment out the `llm_evaluate` in the execution list. You can also specify the models you wish to evaluate. We use the OpenAI API to access the openrouter models, and you can change them to any models of your choice. OpenRouter offers about 275 models, including free options limited to approximately 200 calls per day. To avoid this limitation, you will need to create a paid account to access the non-free models. We have evaluated the free models to test the code and ensure everything works as expected.

Regarding the `keepVectorDB` setting, you can set it to true to prevent re-inputting data if you have already done so. Please note that we input the data from `linguistic_benchmark.json`. Feel free to customize this file for your purposes, as it serves as the source of truth for our evaluator and acts as the answer key for grading the "student."

Additionally, concerning folder directory structures, you can hard-code the date to maintain a consistent directory structure.



In [2]:

execution_steps = [
        "llm_evaluate",
        "generate_statistics",
    ]

local_student = "llm"
openrouter_models = ["google/learnlm-1.5-pro-experimental:free","meta-llama/llama-3.2-11b-vision-instruct:free",
                        "nousresearch/hermes-3-llama-3.1-405b:free","qwen/qwen-2-7b-instruct:free","microsoft/phi-3-medium-128k-instruct:free"]
answer_rounds = 2 # Number of rounds of questions to ask each model
benchmark_name = "Bernard"
# Change to false if you want a new vector db
keepVectorDB = False
# date_now="2024-11-30"  # datetime.now().strftime('%Y-%m-%d')
date_now = datetime.now().strftime('%Y-%m-%d')
folder_name = f"{date_now}-{benchmark_name}"

auto_eval_save_path = f"./{folder_name}/auto_eval_outputs"
stats_save_path = f"./{folder_name}/tables_and_charts"

In [ ]:
for model in openrouter_models:
    model_path = model
    lab, student_models = extract_model_parts(model)
    if student_models:
        print(f"Extracted Lab name: {lab}")

        print(f"Extracted model name: {student_models}")
    else:
        print("Model name not found.")
    student_models = [student_models]

    VectorDB_Controller(keepVectorDB)


    if "llm_evaluate" in execution_steps:
        print("1. GETTING BERNARD LLM Evaluator ANSWERS")
        for n in range(answer_rounds):
            print(f"\n----- Round: {n+1} of {answer_rounds} -----")
            answer_save_path_round = f"{auto_eval_save_path}"

            Bernard_Controller(
                model_path,
                lab,
                student_models,
                answer_save_path_round=answer_save_path_round,
                count=n,
                prefix_replace="auto_eval-",
            )

if "generate_statistics" in execution_steps:
    sub_eval_folders = [f"/round_{r+1}" for r in range(answer_rounds)]

    print("2. GENERATING STATISTICS")
    all_stats_dfs = {}
    save_info = [
        {
            "path": auto_eval_save_path,
            "chart_title": "LLM Linguistic Benchmark Performance",
            "type": "",
        }
    ]
    for info in save_info:
        save_path = info["path"]
        chart_title = info["chart_title"]
        info_type = info["type"]
        print("Eval for path:", save_path)
        all_llm_evals = load_all_llm_answers_from_json(
            save_path,
            prefix_replace="auto_eval-",
            sub_folders=sub_eval_folders,
        )
        stats_df = get_llm_stats(
            all_llm_evals, stats_save_path, file_suffix=info_type, bootstrap_n=10000
        )

        display(stats_df)

        barplot, plt = create_performance_chart(
            stats_df.reset_index(),
            chart_title,
            highlight_models=["o1-preview"],
        )
        barplot.figure.savefig(f"{stats_save_path}/performance_chart{info_type}.png")
        plt.show()
        all_stats_dfs[chart_title] = stats_df

    print("-- DONE STATS --\n")

Additional Charts 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read data from CSV file
df = pd.read_csv(f'{stats_save_path}\\final_stats.csv')

# Sorting DataFrame by Mean Score in descending order for better visualization
df_sorted = df.sort_values(by='mean_score', ascending=False)

# Color palette from the provided PDF
colors = {
    'blue_200': '#90caf9',
    'yellow_600': '#fdd835',
    'pink_200': '#f48fb1',
    'cyan_200': '#80deea',
    'orange_400': '#ffa726',
    'deep_purple_A100': '#b388ff',
    'red_700': '#d32f2f'
}

# Horizontal Bar Chart for Mean Score, CI Lower, and CI Upper for Each Model (Sorted in Descending Order)
y = np.arange(len(df_sorted['model']))  # the label locations
height = 0.25  # the height of the bars

fig, ax = plt.subplots(figsize=(14, 10))
bars1 = ax.barh(y - height, df_sorted['mean_score'], height, label='Mean Score', color=colors['blue_200'])
bars2 = ax.barh(y, df_sorted['ci_lower'], height, label='CI Lower', color=colors['yellow_600'])
bars3 = ax.barh(y + height, df_sorted['ci_upper'], height, label='CI Upper', color=colors['cyan_200'])

# Adding labels and title
ax.set_yticks(y)
ax.set_yticklabels(df_sorted['model'])  # Labels on the left
ax.set_xlabel('Scores')
ax.set_title('Comparison of Mean Score, CI Lower, and CI Upper for Each Model')
ax.invert_yaxis()  # Higher values at the top
ax.legend()

plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Horizontal Bar Chart for Z Interval Error for Each Model (Sorted in Descending Order)
fig, ax = plt.subplots(figsize=(14, 10))
bars = ax.barh(df_sorted['model'], df_sorted['z_interval_error'], color=colors['pink_200'])

plt.ylabel('Models')
plt.xlabel('Z Interval Error')
plt.title('Z Interval Error for Each Model')
ax.invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Horizontal Bar Chart for Mean Score of Each Model (Sorted in Descending Order)
fig, ax = plt.subplots(figsize=(14, 10))
bars = ax.barh(df_sorted['model'], df_sorted['mean_score'], color=colors['orange_400'])

plt.ylabel('Models')
plt.xlabel('Mean Score')
plt.title('Mean Score for Each Model')
ax.invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Plotting Mean Score with Error Bars for Confidence Intervals (Sorted in Descending Order)
ci_error = (df_sorted['ci_upper'] - df_sorted['ci_lower']).abs() / 2
plt.figure(figsize=(14, 10))
plt.errorbar(df_sorted['mean_score'], df_sorted['model'], 
             xerr=ci_error, 
             fmt='o', ecolor=colors['red_700'], capsize=5, label='Mean Score with CI')
plt.ylabel('Models')
plt.xlabel('Mean Score')
plt.title('Mean Score with Confidence Intervals for Various Models')
plt.gca().invert_yaxis()  # Higher values at the top
plt.tight_layout()
plt.legend()
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show()

# Bar Chart of Standard Deviations for Each Model

# Create a bar chart where each model is represented individually
fig, ax = plt.subplots(figsize=(12, 6))

# Plotting standard deviation scores for each model
ax.bar(df['model'], df['std_dev_score'], color='#90caf9', edgecolor='black')

# Adding labels and title
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.xlabel('Model')
plt.ylabel('Standard Deviation')
plt.title('Standard Deviation for Each Model')
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()



## Token Analysis
This provides a straightforward measure of the tokens used per category across all models in a specific run.


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Base directory containing rounds (e.g., 'auto_eval_save_path')
auto_eval_save_path = auto_eval_save_path

# Directory to save the output files
charts_dir = stats_save_path

os.makedirs(charts_dir, exist_ok=True)

# Number of rounds
answer_rounds = 2  # Update as needed

# Function to collect all JSON file paths in a directory
def collect_json_files(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')]

# Function to process JSON files
def process_json_files(file_paths):
    results = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        for _, entry in data.items():
            # Token calculation for each category
            question_tokens = count_tokens(entry.get("question", ""))
            human_answer_tokens = count_tokens(entry.get("human_answer", ""))
            model_answer_input_tokens = count_tokens(entry.get("model_answer", ""))
            eval_response_tokens = count_tokens(entry.get("eval_response", ""))
            score_tokens = count_tokens(str(entry.get("score", "")))
            bernard_evaluator_response_tokens = count_tokens(entry.get("bernard_evaluator_response", ""))
            
            results.append({
                "question_tokens": question_tokens,
                "human_answer_tokens": human_answer_tokens,
                "model_answer_input_tokens": model_answer_input_tokens,
                "eval_response_tokens": eval_response_tokens,
                "score_tokens": score_tokens,
                "bernard_evaluator_response_tokens": bernard_evaluator_response_tokens,
                "total_tokens": question_tokens + human_answer_tokens + model_answer_input_tokens +
                                eval_response_tokens + score_tokens + bernard_evaluator_response_tokens
            })
    return results

# Function to calculate tokens based on the rule: 1 token = 4 characters
def count_tokens(text):
    return max(1, len(text) // 4)

# Process files for each round
all_results = []

for round_num in range(1, answer_rounds + 1):
    round_dir = os.path.join(auto_eval_save_path, f'round_{round_num}')
    
    # Collect files from the round
    json_files = collect_json_files(round_dir)

    # Process the files in the round
    all_results.extend(process_json_files(json_files))

# Convert results to DataFrame for analysis
df = pd.DataFrame(all_results)

# Summarize total tokens per category for comparison
summary = df.sum()

# Create a token usage comparison DataFrame
categories = ["Question", "Human Answer", "Student Response", "Eval Response", "Score",  "Total"]
token_usage = [
    summary["question_tokens"],
    summary["human_answer_tokens"],
    summary["model_answer_input_tokens"],
    summary["eval_response_tokens"],
    summary["score_tokens"],
    summary["total_tokens"]
]

# Create a DataFrame for the results
usage_df = pd.DataFrame({
    "Category": categories,
    "Token Usage": token_usage
})

# Save the token comparison table to a CSV file
usage_csv_path = os.path.join(charts_dir, 'token_usage_comparison.csv')
usage_df.to_csv(usage_csv_path, index=False)

# Create a bar chart for token usage comparison
x = np.arange(len(categories))

# Plot Token Usage
width = 0.35  # Width of the bars
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(x, token_usage, width, label="Token Usage", color="#4C72B0")

# Add values above the bars
for bar in bars:
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5, f"{int(bar.get_height())}", ha="center", fontsize=10)

# Adjust the y-axis dynamically
max_value = max(token_usage)
ax.set_ylim(0, max_value * 1.2)  # Add 20% headroom above tallest bar

# Add labels, title, and legend
ax.set_ylabel("Token Count (Approx)", fontsize=12)
ax.set_title("Token Usage Comparison for Question-Answer Pairs", fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Save the chart as a PNG file
chart_path = os.path.join(charts_dir, 'token_usage_comparison_chart.png')
plt.savefig(chart_path, bbox_inches='tight')
plt.show()

